# Dolo

[Installation instructions](https://github.com/EconForge/dolo/wiki/Installation) for dolo involve a number of dependencies, including the dolo language "dolang."  This notebook assumes all these have been installed.

In [1]:
import dolang
from dolo import *
import BufferStockParameters

Dolo defines models using "model files" whose syntax is specified in the documentation

In [2]:
import sys
import os
os.getcwd()

'/Volumes/Data/Code/ARK/DARKolo/chimeras/BufferStock'

In [3]:
model_dolo = yaml_import("./bufferstock.yaml")
print( model_dolo )

    calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



        Model:
        ------
        name: "anonymous"
        type: "dtcc"
        file: "./bufferstock.yaml

Equations:
----------

transition
 1  : 1.7554 : m(0) == exp(tran(0)) + ((m(-(1))) - (c(-(1)))) * ((R) / ((Γ) * (exp(perm(0)))))

arbitrage
 1  : -0.8726 : (((R) * (β)) * (((((c(1)) * (exp(perm(1)))) * (Γ)) / (c(0))) ** (-(ρ)))) - (1)

definitions




## Explaining the Dolo Model File 

Each item in the `bufferstock.yaml` file corresponds to an equation in the formal mathematics of the BufferStockTheory problem.

### arbitrage:

The first such item we want to consider is labelled `arbitrage`.  In Dolo, the arbitrage equation represents a condition that should be equal to zero for an optimizing agent (perhaps it should be called the `no-arbitrage` equation since the idea is that the amount by which you could make yourself better is zero).  

See the dolo/dolark documentation to absorb syntactic conventions like the fact that time offsets are signified by parens; e.g., `c(-1)` is the variable `c` one period in the past.

If the model is stochastic, equations that involve a future-dated variable are calculated in expectation.

Dolo's specification of the model defines `perm` as the logarithm of what is called $\psi$, so its exponential `exp(perm(1))` is equivalent to $\psi_{t+1}$ in the companion notebook's notation.  Finally, the expression `| 0.0 <= c <= m` corresponds to the liquidity constraint mentioned above (mathematically, $0 \leq c \leq m$). Thus, the arbitrage equation:

    arbitrage:
        - (R*β*((c(1)*exp(perm(1))*Γ)/c)^(-ρ)-1 ) | 0.0<=c<=m

is equivlent to the Euler equation in the HARK notebook, because:

\begin{align*}
0 & = R \beta \mathbb{E}_{t}[(\psi c_{t+1}\Gamma /c_{t})^{-\rho})]-1 \\ 
1 & = R \beta \mathbb{E}_{t}[(\Gamma \psi \frac{c_{t+1}}{c_{t}})^{-\rho})] 
\\c_{t}^{-\rho} & = R \beta \mathbb{E}_{t}[(\Gamma \psi c_{t+1})^{-\rho})] \\
\end{align*}

### transition:

The second equation in the `dolang` model is:

    transition:
        - m = exp(tran) + (m(-1)-c(-1))*(R/(Γ*exp(perm)))

Recall that the dynamic budget constraints in the HARK problem are:
\begin{eqnarray*}
a_t &=& m_t - c_t \\
m_{t+1} &=& R/(\Gamma \psi_{t+1}) a_t + \theta_{t+1} \\
\end{eqnarray*}

Substituting the first condition, defining $a_t$, into the second, and rearranging, we get:
\begin{eqnarray}
m_{t+1} &=& \theta_{t+1} +  (m_t - c_t) R/(\Gamma \psi_{t+1})
\end{eqnarray}
and $\theta_{t+1}$ is equivalent to the exponential of the lognormally-distributed stochastic variable `tran`.  (In dolo, the equation's timing is shifted back a period because its expectation is not being taken)

In [5]:
# Set a maximum value of the market resources ratio m for use in both models
max_m = BufferStockParameters.Common['max_m']
model_dolo.data['calibration']['max_m'] = max_m

# Obtain the decision rule by time iteration
dr = time_iteration(model_dolo,tol=1e-08,verbose=True)

------------------------------------------------
| N   |  Error     | Gain     | Time     | nit |
------------------------------------------------
|   1 |  9.000e-01 |      nan |    0.176 |   5 |
|   2 |  2.622e-02 |    0.029 |    0.129 |   5 |
|   3 |  1.962e-02 |    0.748 |    0.124 |   5 |
|   4 |  1.979e-02 |    1.009 |    0.128 |   5 |
|   5 |  2.098e-02 |    1.060 |    0.129 |   5 |
|   6 |  2.335e-02 |    1.113 |    0.141 |   5 |
|   7 |  2.546e-02 |    1.090 |    0.139 |   5 |
|   8 |  2.664e-02 |    1.046 |    0.086 |   3 |
|   9 |  2.575e-02 |    0.966 |    0.075 |   3 |
|  10 |  2.378e-02 |    0.924 |    0.075 |   3 |
|  11 |  2.534e-02 |    1.066 |    0.077 |   3 |
|  12 |  2.549e-02 |    1.006 |    0.092 |   3 |
|  13 |  2.491e-02 |    0.977 |    0.079 |   3 |
|  14 |  2.588e-02 |    1.039 |    0.077 |   3 |
|  15 |  2.537e-02 |    0.980 |    0.082 |   3 |
|  16 |  2.618e-02 |    1.032 |    0.079 |   3 |
|  17 |  2.620e-02 |    1.001 |    0.079 |   3 |
|  18 |  2.637e-02 |